In [23]:
from data_utils import get_data_loaders, prepare_train, get_cv_data_loaders, prepare_test
from engine import training_step, validation_step
from utils import print_trainable_parameters
from model import ClsModel1, save_model, load_model

import pandas as pd
import torch
from torch import nn

In [24]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(0)
model  = ClsModel1(input_size=21, hidden_size=100,num_classes=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

trainable params: 103,301 || all params: 103,301 || trainable%: 100.0


In [25]:
df=pd.read_csv("data/train.csv", index_col='id')
X_train, y_train, scaler = prepare_train(df)
test = pd.read_csv("data/test.csv", index_col='id')
test = prepare_test(test, scaler)

In [26]:
import os

def predict_with_models(test, directory ,model, optimizer):
    predictions = {}
    model_files = [f for f in os.listdir(directory) if f.endswith('.pt')]
    model_files.sort()
    for model_file in model_files:
        model_name = os.path.splitext(model_file)[0]
        model, optimizer, epoch, loss, scheduler  = load_model(model,optimizer, os.path.join(directory, model_file))
        model = model.to(device)
        with torch.inference_mode():
            preds = model(torch.tensor(test,device=device,dtype=torch.float64))
            preds = preds.cpu().detach().numpy()
        predictions[model_name] = preds.squeeze()
    return pd.DataFrame(predictions)

In [27]:
# preds_df = predict_with_models(test, 'saved_models', model, optimizer)

In [28]:
from model import ClsModel2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(0)
model  = ClsModel2(input_size=21, hidden_size=50,num_classes=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
model, optimizer, epoch, loss, scheduler  = load_model(model,optimizer, "model_all_data.pt")
with torch.inference_mode():
            preds = model(torch.tensor(test,device=device,dtype=torch.float64))
            preds = preds.cpu().detach().numpy()
preds

trainable params: 16,901 || all params: 16,901 || trainable%: 100.0


array([[0.61329224],
       [0.3278715 ],
       [0.2710264 ],
       ...,
       [0.4497009 ],
       [0.0730841 ],
       [0.06233596]])

In [29]:
submission = pd.read_csv("data/sample_submission.csv", index_col='id')
submission['smoking'] = preds
submission.to_csv('data/submission.csv')

In [30]:
submission.isna().sum()

smoking    0
dtype: int64